In [ ]:
import subprocess
import time
import datetime
from twilio.rest import Client
from twilio_info import *
import logging 
import google.cloud.logging
import re


# Setup Logging
file_name = ("Ping Test")
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(filename)s:%(lineno)s - %(funcName)s() %(message)s")
logger.setLevel(logging.INFO)

auth_file = "VirtualToolbox-5f6ecffdcf58.json"

client = google.cloud.logging.Client.from_service_account_json(auth_file)
client.setup_logging()


# Setup Twilio

client = Client(twilio_sid, twilio_auth_token)

def send_alert(msg):
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    msg_body = "{}:  {}".format(dtnow, msg)
    try:
        logger.error(msg_body)
        message = client.messages.create(body=msg_body,from_=phone_num_from,to=phone_num_to)
        time.sleep(30)
        logger.info("Attemped to send alert text.  Status:  {}".format(message.status))
    except:
        logger.critical("Attemped to send alert text.  Status:  Failed")
 

# Setup Ping script

def ping_addr(ip_address):
    try:
        res = subprocess.call(['ping', '-c', '3', ip_address])
    except:
        res = 99
    dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
    if res == 0:
        print ("{} - ping to {} OK".format(dtnow, ip_address))
    elif res == 2:
        print ("{} - no response from {}".format(dtnow, ip_address))
    else:
        print ("{} - ping to {} failed!".format(dtnow, ip_address))
    return(res)
    

def ping_sweep(subnet):
    for x in range(1, 255):
        ip_address = f"{subnet}{x}"
        print(ip_address)
        subprocess.call(['ping', '-w', '0.1', '-c', '1', ip_address])
    return    

class address:
    def __init__(self, ip_address, mac_address):
        self.call_count = 0
        self.ip_address = ip_address
        self.mac_address = mac_address
        if not self.verify_address(ip_address, mac_address):
            ip_address = self.find_ip_address(mac_address, subnet)
            if len(ip_address) > 6:
                self.ip_address = ip_address
    
    def find_mac_address(self, ip_address):
        ping_addr(ip_address)
        pid = subprocess.Popen(["arp", "-n", ip_address], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        results = re.search(r"(([a-f\d]{1,2}\:){5}[a-f\d]{1,2})", str(s))
        if results:
            mac = results.groups()[0]
        else:
            mac = ''
        return(mac)

    def find_ip_address(self, mac_address, subnet):
        ping_sweep(subnet)
        pid = subprocess.Popen(["arp", "-an"], stdout=subprocess.PIPE)
        s = pid.communicate()[0]
        results = re.search(f"\(([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)\) at {mac_address.lower()}", str(s).lower())
        if results:
            ip_address = results[1]
        else:
            ip_address = ''
        return(ip_address)

    def verify_address(self, ip_address, mac_address):
        mac_address2 = self.find_mac_address(ip_address)
        return(mac_address.lower() == mac_address2.lower())
    
    def get_ip_address(self):
        self.call_count += 1
        if self.call_count > 10:
            self.__init__(self.ip_address, self.mac_address)
        return(self.ip_address)
    
def main(ip_address, mac_address, device_name):    
    addr = address(ip_address, mac_address)
    alert_active = False
    while True:
        alert_count = 0

        while alert_count < 4:
            status = ping_addr(addr.get_ip_address())
            count = 0
            
            if (alert_active)&(status == 0):
                dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
                print("{} - Host back up".format(dtnow))
                send_alert("{} has been restored!".format(device_name))
                alert_active = False
                

            while (status != 0)&(count<5):
                dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
                print("{} - Ping failed, backoff and retry".format(dtnow))
                time.sleep(3)
                status = ping_addr(ip_address)
                count += 1

            if status != 0:
                send_alert("{} monitor is not responding!".format(device_name))
                alert_count += 1
                alert_active = True

            time.sleep(30)

        if alert_count >= 4:
            dtnow = datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d %H:%M")
            print("{} - Exceeded alert count, backing off for 4 hours".format(dtnow))
            time.sleep(4*60*60)
            count = 0
            alert_count = 0

logger.info("Starting Ping Test")



ip_address = "10.0.2.217"
subnet = '10.0.2.'
mac_address = '58:EF:68:EA:54:2D'
device_name = 'Freezer'

main(ip_address, mac_address, device_name)


<ipython-input-1-f29b4bd6517c>:143 - <module>() Starting Ping Test
Starting Ping Test


2022-08-18 16:55 - no response from 10.0.2.217
2022-08-18 16:55 - no response from 10.0.2.217
2022-08-18 16:55 - Ping failed, backoff and retry
2022-08-18 16:56 - no response from 10.0.2.217
2022-08-18 16:56 - Ping failed, backoff and retry
2022-08-18 16:56 - no response from 10.0.2.217
2022-08-18 16:56 - Ping failed, backoff and retry
2022-08-18 16:56 - no response from 10.0.2.217
2022-08-18 16:56 - Ping failed, backoff and retry
2022-08-18 16:56 - no response from 10.0.2.217
2022-08-18 16:56 - Ping failed, backoff and retry


<ipython-input-1-f29b4bd6517c>:31 - send_alert() 2022-08-18 16:57:  Freezer monitor is not responding!
2022-08-18 16:57:  Freezer monitor is not responding!
http_client.py:73 - request() POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
http_client.py:75 - request() PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2022-08-18 16:57:  Freezer monitor is not responding!'}
PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2022-08-18 16:57:  Freezer monitor is not responding!'}


2022-08-18 16:57 - no response from 10.0.2.217


http_client.py:91 - request() POST Response: 201 {"body": "2022-08-18 16:57:  Freezer monitor is not responding!", "num_segments": "1", "direction": "outbound-api", "from": "+12052933447", "date_updated": "Thu, 18 Aug 2022 22:57:10 +0000", "price": null, "error_message": null, "uri": "/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages/SMbf27690334af381d43c70c725a0ec462.json", "account_sid": "AC4ec8a19b2be6797eb5825903ff609d39", "num_media": "0", "to": "+13038182403", "date_created": "Thu, 18 Aug 2022 22:57:10 +0000", "status": "queued", "sid": "SMbf27690334af381d43c70c725a0ec462", "date_sent": null, "messaging_service_sid": null, "error_code": null, "price_unit": "USD", "api_version": "2010-04-01", "subresource_uris": {"media": "/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages/SMbf27690334af381d43c70c725a0ec462/Media.json"}}
POST Response: 201 {"body": "2022-08-18 16:57:  Freezer monitor is not responding!", "num_segments": "1", "direction": "outbound-api"

...

<ipython-input-1-f29b4bd6517c>:31 - send_alert() 2022-08-18 20:05:  Freezer monitor is not responding!
2022-08-18 20:05:  Freezer monitor is not responding!
http_client.py:73 - request() POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
POST Request: https://api.twilio.com/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages.json
http_client.py:75 - request() PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2022-08-18 20:05:  Freezer monitor is not responding!'}
PAYLOAD: {'To': '13038182403', 'From': '12052933447', 'Body': '2022-08-18 20:05:  Freezer monitor is not responding!'}


2022-08-18 20:05 - no response from 10.0.2.217


http_client.py:91 - request() POST Response: 201 {"body": "2022-08-18 20:05:  Freezer monitor is not responding!", "num_segments": "1", "direction": "outbound-api", "from": "+12052933447", "date_updated": "Fri, 19 Aug 2022 02:05:48 +0000", "price": null, "error_message": null, "uri": "/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages/SM7a2f6c3b9e90471ba334b823f0fe8a13.json", "account_sid": "AC4ec8a19b2be6797eb5825903ff609d39", "num_media": "0", "to": "+13038182403", "date_created": "Fri, 19 Aug 2022 02:05:48 +0000", "status": "queued", "sid": "SM7a2f6c3b9e90471ba334b823f0fe8a13", "date_sent": null, "messaging_service_sid": null, "error_code": null, "price_unit": "USD", "api_version": "2010-04-01", "subresource_uris": {"media": "/2010-04-01/Accounts/AC4ec8a19b2be6797eb5825903ff609d39/Messages/SM7a2f6c3b9e90471ba334b823f0fe8a13/Media.json"}}
POST Response: 201 {"body": "2022-08-18 20:05:  Freezer monitor is not responding!", "num_segments": "1", "direction": "outbound-api"

2022-08-18 20:07 - Exceeded alert count, backing off for 4 hours


Failed to submit 1 logs.
Traceback (most recent call last):
  File "/Users/jefalexa/GitHub/ping_monitor/env/lib/python3.8/site-packages/google/api_core/grpc_helpers.py", line 57, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "/Users/jefalexa/GitHub/ping_monitor/env/lib/python3.8/site-packages/grpc/_channel.py", line 824, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/Users/jefalexa/GitHub/ping_monitor/env/lib/python3.8/site-packages/grpc/_channel.py", line 726, in _end_unary_response_blocking
    raise _InactiveRpcError(state)
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.DEADLINE_EXCEEDED
	details = "Deadline Exceeded"
	debug_error_string = "{"created":"@1660875473.246253000","description":"Deadline Exceeded","file":"src/core/ext/filters/deadline/deadline_filter.cc","file_line":69,"grpc_status":4}"
>

The above exception was the direct cause of the following except